$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 3: Neural Network Regression

Bradley Pospeck

## Overview

Compare linear and neural network models applied to a data set of your choice.  

## Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Required Code

Download [nnA3.tar](http://www.cs.colostate.edu/~anderson/cs480/notebooks/nnA3.tar) and extract its contents, which are

* `neuralnetworks.py`
* `scaledconjugategradient.py`
* `mlutils.py`

as discussed in lecture. 

Write the following functions that train and evaluate linear and neural network models.

* `model = trainLinear(X,T,parameters)`
* `error = evaluateLinear(model,X,T)`
* `model = trainNN(X,T,parameters)`
* `error = evaluateNN(model,X,T)`

Write a new and improved version of the function we have been using to partition data into folds and combine them into training, validation and testing subsets.  Call this new version `trainValidateTestKFolds`.  Instead of returning data subsets, it applies train and evaluate functions to the subsets.  It uses the validation subset to determine the best model parameter values.  It must implement the following algorithm. The part up to "COMPLETE THIS FUNCTION..." is given in Python.  Do not change it.  Do write the rest of the function.

```
def trainValidateTestKFolds(trainf,evaluatef,X,T,parameterSets,nFolds,
                            shuffle=False,verbose=False):
    # Randomly arrange row indices
    rowIndices = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(rowIndices)
    # Calculate number of samples in each of the nFolds folds
    nSamples = X.shape[0]
    nEach = int(nSamples / nFolds)
    if nEach == 0:
        raise ValueError("partitionKFolds: Number of samples in each fold is 0.")
    # Calculate the starting and stopping row index for each fold.
    # Store in startsStops as list of (start,stop) pairs
    starts = np.arange(0,nEach*nFolds,nEach)
    stops = starts + nEach
    stops[-1] = nSamples
    startsStops = list(zip(starts,stops))
    # Repeat with testFold taking each single fold, one at a time
    results = []

    # COMPLETE THIS FUNCTION BY IMPLEMENTING THE FOLLOWING STEPS.
    
    # For each test fold
        for parmSet in parameterSets:  # For each set of parameter values, called parmSet
            # Find best set of parameter values
            # For each validate fold (except when same as test fold)
                # Use trainf to fit model to training data using parmSet
                # Calculate the error of this model by calling evaluatef with 
                #  the model and validation data
            # Calculate the mean of these errors.
            # If this error is less than the previously best error for parmSet, 
            # update best parameter values and best error
        # Make a new set of training data by concatenating the training and 
        # validation data from previous step.
        # Retrain, using trainf again, to fit a new model to this new training data.
        # Calculate error of this new model on the test data, and also on the new 
        #  training data.
        # Construct a list of the best parameter values with this training error, 
        #  the mean of the above valdiation errors, and the testing error
        # Print this list if verbose == True
        # Append this list to a result list
    # Return this result list
``` 

The `parameters` argument to the `trainLinear` function is just the value of $\lambda$. For the `trainNN` function it must specify the hidden layer structure and the number of Scaled Conjugate Gradient iterations.  Here are some examples:
```
model = trainNN(X,T,[5, 100])       # Single hidden layer of 5 units, trained 
                                    # for 100 iterations
model = trainNN(X,T,[[10,10], 200]) # Two hidden layers, 10 units each, 
                                    # trained for 200 iterations
```

In [3]:
def trainValidateTestKFolds(trainf,evaluatef,X,T,parameterSets,nFolds,
                            shuffle=False,verbose=False):
    # Randomly arrange row indices
    rowIndices = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(rowIndices)
    # Calculate number of samples in each of the nFolds folds
    nSamples = X.shape[0]
    nEach = int(nSamples / nFolds)
    if nEach == 0:
        raise ValueError("partitionKFolds: Number of samples in each fold is 0.")
    # Calculate the starting and stopping row index for each fold.
    # Store in startsStops as list of (start,stop) pairs
    starts = np.arange(0,nEach*nFolds,nEach)
    stops = starts + nEach
    stops[-1] = nSamples
    startsStops = list(zip(starts,stops))
    # Repeat with testFold taking each single fold, one at a time
    results = []

    # COMPLETE THIS FUNCTION BY IMPLEMENTING THE FOLLOWING STEPS.
    
    # For each test fold
        for parmSet in parameterSets:  # For each set of parameter values, called parmSet
            # Find best set of parameter values
            # For each validate fold (except when same as test fold)
                # Use trainf to fit model to training data using parmSet
                # Calculate the error of this model by calling evaluatef with 
                #  the model and validation data
            # Calculate the mean of these errors.
            # If this error is less than the previously best error for parmSet, 
            # update best parameter values and best error
        # Make a new set of training data by concatenating the training and 
        # validation data from previous step.
        # Retrain, using trainf again, to fit a new model to this new training data.
        # Calculate error of this new model on the test data, and also on the new 
        #  training data.
        # Construct a list of the best parameter values with this training error, 
        #  the mean of the above valdiation errors, and the testing error
        # Print this list if verbose == True
        # Append this list to a result list
    # Return this result list

IndentationError: unexpected indent (<ipython-input-3-0755e613f270>, line 24)

## Examples

```
X = np.arange(100).reshape((-1, 1))
T = np.abs(X -50) + X

result = trainValidateTestKFolds(trainLinear, evaluateLinear ,X, T, range(0,101,10), nFolds=5, shuffle=False)
                                 
print('Linear Model\n{:^10s} {:>10s} {:>10s} {:>10s}'.format('lambda', 'train', 'validate', 'test RMSE'))
for x in result:
    print('{:^10.2f} {:>10.3f} {:10.3f} {:10.3f}'.format(*x))
```
should result in
```
Linear Model
  lambda        train   validate  test RMSE
  30.00        13.576     16.028     19.301
  100.00       22.438     20.407     20.099
  30.00        14.419     16.542     25.194
  70.00        21.098     24.589      9.615
  100.00       13.036     12.768     55.356
```

An example of fitting a neural network to this data is
```
import itertools
parms = list(itertools.product([2, 5, 10, 20, [5, 5], [10, 2, 10]], [10, 20, 100, 500]))
result = trainValidateTestKFolds(trainNN, evaluateNN, X, T, parms, nFolds=5, shuffle=False)
print('NN Model\n{:>30s} {:>10s} {:>10s} {:>10s}'.format('(Hidden Units, Iterations)', 'train', 'validate', 'test RMSE'))
for x in result:
    print('{:>30s} {:10.3f} {:10.3f} {:10.3f}'.format(str(x[0]), *x[1:]))
```
should result in something like 
```
NN Model
    (Hidden Units, Iterations)      train   validate  test RMSE
            ([10, 2, 10], 500)      0.289      1.751      0.319
                 ([5, 5], 500)      0.150      2.066      0.259
                     (20, 500)      0.045      1.174      2.584
                     (20, 500)      0.250      7.903      1.258
                     (20, 500)      0.162      2.330      5.648
```
Your results will differ, due to the initial random values for the neural network's weights.

## Data

Pick a data set that is interesting to you.  Explain why you picked it.  Investigate the data with some plots and describe what you see.

## Results

Investigate the use of a linear model with various $\lambda$ values for your chosen data.  Then investigate the use of a neural network model on the same data for various sizes of hidden layers and numbers of iterations.
Discuss the results obtained, including the error values and the parameter values that result in the best performance.

Apply the `trainValidateTestKFolds` function, print the results, and discuss them.  How much variation is there in the best parameter values across different test folds?  Does the complexity of the models, in terms of the best parameters found, make sense?

## Grading

Your notebook will be run and graded automatically. Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs480/notebooks/A3grader.tar) and extract `A3grader.py` from it. Run the code in the following cell to demonstrate an example grading session.  You should see a perfect score of 80/100 if your functions are defined correctly.  The remaining 20% will be based on the instructors reading of your notebooks.  We will be looking for how well the method is explained in text with some LaTeX math, and how well the results are summarized.

In [8]:
%run -i "A3grader.py"

 Testing: result = trainValidateTestKFolds(trainLinear,evaluateLinear,X,T,
                  range(0,101,10),nFolds=5,shuffle=False)
 Result is
    10   3.158   3.306   2.414
    20   4.368   4.017   3.641
    10   3.245   3.343   5.03
    20   4.448   4.856   2.024
    20   2.426   2.377   10.89
20/20 points. First column, of best lambda values, is correct.
20/20 points. Columns of RMSE values are correct.
 Running:
   import itertools
   parms = list(itertools.product([[5],[5,5],[2,2,2]], [10,50,100,200]))
   te = []
   for rep in range(5):
       result = trainValidateTestKFolds(trainNN,evaluateNN,X,T,
                                        parms,
                                        nFolds=4,shuffle=False)
       resulte = np.array([r[1:] for r in result])
       meanTestRMSE = resulte[:,-1].mean()
       print('     ',meanTestRMSE)
       te.append(meanTestRMSE)
      2.10077032973
      2.00326657671
      2.15693009859
      2.48018047289
      1.15109968332
40/40 points. Me

## Check-in

Do not include this section in your notebook.

Name your notebook ```Lastname-A3.ipynb```.  So, for me it would be ```Anderson-A3.ipynb```.  Submit the file using the ```Assignment 3``` link on [Canvas](https://colostate.instructure.com/courses/41327).

Grading will be based on 

  * correct behavior of the required functions,
  * readability of the notebook,
  * effort in making interesting observations, and in formatting your notebook.